In [1]:
import pandas as pd
import json
import numpy as np
import os
from pathlib import Path
import math
from torchvision import datasets, transforms

In [2]:
path_ = 'dataset_idx/cifar10/noniid/CIFAR10_30client_pareto.json'
with open(path_, 'r') as f:
    data_idx = json.load(f)
data_idx


{'0': [23355,
  21165,
  30123,
  7440,
  31530,
  19502,
  3067,
  45674,
  47090,
  49301,
  4626,
  38373,
  34059,
  11375,
  42425,
  10248,
  36611,
  8801,
  49875,
  26712,
  41963],
 '1': [10457, 25564, 34960, 30136, 2615, 23160, 3454, 40163, 32587],
 '2': [5424, 27315, 43258],
 '3': [34102,
  44688,
  14285,
  1180,
  11221,
  25426,
  48089,
  46877,
  41208,
  9101,
  43162,
  3796,
  4700,
  10976,
  11500,
  32816,
  15940,
  36920,
  15505,
  41947,
  10289,
  12346,
  8543,
  40480,
  31285,
  39536,
  33936,
  48523,
  7684,
  4236,
  32343,
  13409,
  14333,
  39949,
  28429,
  32561,
  30417,
  29272,
  23204,
  10232,
  42338,
  27763,
  22653,
  46167,
  4157,
  847,
  28446,
  3104,
  43168,
  46996,
  31985,
  22358,
  815,
  37468,
  22396,
  30492,
  34616,
  49151,
  997,
  36236,
  34145,
  41133,
  6130,
  33751,
  35531,
  18331,
  37296,
  33724,
  7977,
  28059,
  43371,
  30851,
  34420,
  25881,
  35966],
 '4': [10525,
  31702,
  12309,
  26897,
  45786

In [3]:
cifar10_dataset = datasets.CIFAR10(
        root="benchmark/cifar10/data/", 
        # split="letters",
        train=True, 
        download=True
    )

Files already downloaded and verified


In [4]:
labels = cifar10_dataset.targets
idxs = range(len(cifar10_dataset))
idxs_labels = np.vstack((idxs, labels))
idxs_labels = idxs_labels[:, idxs_labels[1, :].argsort()]
idxs = idxs_labels[0, :]
labels = idxs_labels[1, :]

In [5]:
idxs

array([29513, 16836, 32316, ..., 36910, 21518, 25648])

In [6]:
labels

array([0, 0, 0, ..., 9, 9, 9])

In [7]:
list_idx_clean = []
count = 0
for key in data_idx.keys():
    if int(key) in range(15, 30):
        list_idx_clean = list_idx_clean + data_idx[key]
        count += len(data_idx[key])

In [8]:
count

1518

In [9]:
len(list_idx_clean)

1518

In [10]:
idxs_list = idxs.tolist()


In [11]:
list_remove_idx_clean = [ele for ele in idxs_list if ele not in list_idx_clean]

In [12]:
tmp = 0
list_tmp = []
for i in range(len(list_remove_idx_clean)):
    if(cifar10_dataset[list_remove_idx_clean[i]][1] == tmp):
        tmp +=1
        list_tmp.append(i)
list_tmp.append(len(list_remove_idx_clean))

In [13]:
list_tmp

[0, 4912, 9747, 14603, 19494, 24467, 29332, 34243, 39063, 43931, 48489]

In [14]:
new_data_idx = {}
for client in range(30):
    if client in range(15):
        client_list_idx = []
        for class_ in range(10):
            client_list_idx = client_list_idx + list_remove_idx_clean[list_tmp[class_] + client*100: list_tmp[class_] + client*100 +100 ]
        new_data_idx[str(client)] = client_list_idx
    else:
        new_data_idx[str(client)] = data_idx[str(client)]
        

In [15]:
new_data_idx['13']


[34799,
 15884,
 34711,
 34715,
 15887,
 34722,
 15886,
 34724,
 34745,
 34746,
 34948,
 34748,
 34758,
 34761,
 34766,
 7592,
 34780,
 34781,
 34783,
 34787,
 34750,
 34949,
 3382,
 34964,
 3344,
 15678,
 15677,
 35147,
 35149,
 35162,
 3337,
 3336,
 35132,
 3335,
 35179,
 3332,
 35182,
 3329,
 35195,
 35196,
 35201,
 35202,
 3334,
 35248,
 15689,
 35121,
 34989,
 34998,
 3375,
 15767,
 15755,
 35028,
 35039,
 35042,
 3347,
 15735,
 3361,
 35059,
 7631,
 35079,
 15707,
 35089,
 3352,
 35095,
 3362,
 14486,
 29596,
 18073,
 5696,
 20602,
 24404,
 24420,
 20584,
 20574,
 20568,
 24445,
 20566,
 24447,
 5684,
 20555,
 24476,
 20539,
 20537,
 24488,
 5673,
 24395,
 24387,
 5699,
 24383,
 20622,
 24333,
 5719,
 24341,
 6570,
 24349,
 24351,
 24355,
 24491,
 24356,
 24361,
 41215,
 41217,
 41230,
 41241,
 41248,
 41254,
 41257,
 41192,
 40936,
 41092,
 41087,
 40958,
 40964,
 40967,
 40971,
 40972,
 40987,
 41006,
 41091,
 41015,
 41026,
 41032,
 41051,
 41061,
 41063,
 41067,
 41072,
 4107

In [16]:
rs = []
for client in range(30):
    tmp = []
    for i in range(10):
        tmp.append(sum(cifar10_dataset[j][1] == i for j in new_data_idx[str(client)]))
    rs.append(tmp)
df = pd.DataFrame(rs,columns=[f"Label_{i}" for i in range(10)])

In [17]:
df

,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7,Label_8,Label_9
0,100,100,100,100,100,100,100,100,100,100
1,100,100,100,100,100,100,100,100,100,100
2,100,100,100,100,100,100,100,100,100,100
3,100,100,100,100,100,100,100,100,100,100
4,100,100,100,100,100,100,100,100,100,100
5,100,100,100,100,100,100,100,100,100,100
6,100,100,100,100,100,100,100,100,100,100
7,100,100,100,100,100,100,100,100,100,100
8,100,100,100,100,100,100,100,100,100,100
9,100,100,100,100,100,100,100,100,100,100


In [ ]:
df.to_csv('dataset_idx/cifar10/noniid/CIFAR10_30client_pareto_15_attacker_1000sample.csv', index=False, header=False)


In [42]:
def save_dataset_idx(list_idx_sample, path, filename="dataset_idx.json"):
    if not Path(path).exists():
        os.system(f"mkdir -p {path}")
        
    with open(path + filename, "w+") as outfile:
        json.dump(list_idx_sample, outfile)

In [ ]:
save_dataset_idx(new_data_idx, 'dataset_idx/cifar10/noniid/', 'CIFAR10_30client_pareto_15_attacker_1000sample.json')